<a href="https://colab.research.google.com/github/bigyansapkota97/Covid-vs-Stock-Price/blob/main/Python_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alpha_vantage

In [ ]:
import pandas as pd

url_conf_cases = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

df_conf_cases_initial = pd.read_csv(url_conf_cases)
df_deaths_initial = pd.read_csv(url_deaths)

#converting date columns to datetime format
date_columns = df_conf_cases_initial.columns[4:]
date_range = pd.to_datetime(date_columns)

# getting data from February 2020 to August 2021
mask = (date_range >= '2020-02-01') & (date_range <= '2021-08-31')
df_conf_cases = df_conf_cases_initial.iloc[:, :4].join(df_conf_cases_initial.iloc[:, 4:][date_columns[mask]])
df_deaths = df_deaths_initial.iloc[:, :4].join(df_deaths_initial.iloc[:, 4:][date_columns[mask]])

# Global level sum of confirmed cases and deaths
global_confirmed = df_conf_cases.iloc[:, 4:].sum(axis=0)
global_deaths = df_deaths.iloc[:, 4:].sum(axis=0)

# Global aggregated dataframe
global_df = pd.DataFrame({
    'Date': global_confirmed.index,
    'Confirmed Cases': global_confirmed.values,
    'Deaths': global_deaths.values
})

print(global_df)

        Date  Confirmed Cases   Deaths
0     2/1/20            12038      260
1     2/2/20            16787      364
2     2/3/20            19887      428
3     2/4/20            23899      494
4     2/5/20            27644      566
..       ...              ...      ...
573  8/27/21        215988154  4513136
574  8/28/21        216545334  4521572
575  8/29/21        216992164  4528394
576  8/30/21        217664015  4537965
577  8/31/21        218284677  4547280

[578 rows x 3 columns]


In [ ]:
from alpha_vantage.timeseries import TimeSeries

API_Key = 'X1R4RZ0VV0TG8CUE'
ts = TimeSeries(key=API_Key, output_format='pandas')

# Function to retrieve and filter stock data
def get_filtered_stock_data(symbol, start_date, end_date):
    ts = TimeSeries(key=API_Key, output_format='pandas')
    data, meta_data = ts.get_daily(symbol, outputsize='full')
    filtered_data = data.loc[start_date:end_date]
    filtered_data = filtered_data.sort_index()
    return filtered_data[['1. open', '2. high', '3. low', '4. close', '5. volume']]

stocks = ['FSV.TO', 'IBM', 'DAL']

In [ ]:
# Retrieve and append 3 stock data
for stock_symbol in stocks:
    stock_data = get_filtered_stock_data(stock_symbol, '2020-02-01', '2021-08-31')
    stock_data.columns = [f'{col}_{stock_symbol}' for col in stock_data.columns]

    # making same data type 'Date' columns
    stock_data['Date'] = pd.to_datetime(stock_data.index)
    global_df['Date'] = pd.to_datetime(global_df['Date'])

    stock_data = stock_data.reset_index(drop=True)
    global_df = pd.merge(global_df, stock_data, on='Date', how='outer')

global_df = global_df.iloc[:, :18]

# Drop rows with any missing values
global_df = global_df.dropna()

global_df.to_csv('global_data.csv', index=False)

print(global_df)

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and set the size
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# Plot Deaths over Time
ax1.plot(global_df['Date'], global_df['Deaths'], label='Deaths', color='red')
ax1.set_title('Deaths Over Time')
ax1.set_ylabel('Number of Deaths')

# Plotting Confirmed Cases over Time
ax2.plot(global_df['Date'], global_df['Confirmed Cases'], label='Confirmed Cases', color='blue')
ax2.set_title('Confirmed Cases Over Time')
ax2.set_xlabel('Date')
ax2.set_ylabel('Number of Confirmed Cases')

ax1.legend()
ax2.legend()

plt.show()